# Tratamiento de datos

En este notebook se realizará la extracción y el respectivo tratamiento de los dataframes que se usarán en el análisis. Los dataframes que se generarán serán los siguientes
 * [info_countries.csv](data/info_countries.csv): Información variada sobre los países en español.
 * [olympics.csv](data/olympics.csv): Dataframe con la información de las medallas ganadas por equipo olímpico.
El dataframe `olympics.csv` será la base de datos principal para este análisis.

Adicionalmente, también se ha generado un archivo XLSX llamado [olympics_excel.xlsx](data/olympics_excel.xlsx) con información idéntica a la que contiene `olympics.csv`.

## Imports

Aquí se presentan los paquetes que se usarán para el tratamiento de datos.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import os

from unidecode import unidecode
from countryinfo import CountryInfo
from googletrans import Translator
import pycountry

## Fuentes

A continuación se listan las url donde se obtienen los datos

 * Medallas ganadas por país

In [3]:
url_medallas = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"

 * Lista de países por continente

In [4]:
url_paises = "https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_por_continentes"

 * Equivalencias entre códigos DOI y ISO

In [5]:
url_codigos = "https://es.wikipedia.org/wiki/Anexo:Comparaci%C3%B3n_de_los_c%C3%B3digos_del_COI,_la_FIFA_y_la_ISO_3166"

Los dataframes generados se guardarán en el directorio llamado `data/`, bajo los siguientes nombres,
 * `info_countries.csv`: Información variada de los países
 * `olympics.csv`: Información de medallas ganadas por país
 * `olympics_excel.xlsx`: Información de medallas ganadas por país en formato excel

In [6]:
data_path = "data"
if not os.path.exists(data_path):
	os.mkdir(data_path)

## Funciones a utilizar

La siguiente función se usará para darle un formato más cómodo a las columnas del dataframe resultante

## Extracción de los dataframes

### Dataframe de medallas por país

A continuación se extrae la información que Wikipedia posee de las medallas olímpicas ganadas por país, tanto en olimpiadas de verano como en las de invierno.

Primero extraemos de la url correspondiente.

In [7]:
df = pd.read_html(url_medallas)[1]

Renombramos las columnas para que la manipulación del data frame sea más cómodo

In [8]:
columns = ['Country', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']

df.columns = columns
df = df[df["Country"] != "Totals"]

Una columna posee tanto el nombre del país como el código COI, por lo que crearemos dos columnas más donde los separaremos

In [9]:
df["Country_code_coi"] = df.Country.str.extract(r"\((\w{3})\)")[0]
df["Country_name"] = df.Country.str.extract(r"(.*)\s\(")[0]
df["Country"] = df["Country_name"] + " (" + df["Country_code_coi"] + ")"
df = df[['Country', 'Country_name','Country_code_coi', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]

Las columnas del dataframe están listas y son las siguientes
 * `Country`: Nombre del país y código expedido por el Comité Olímpico Internacional.
 * `Country_name`: Nombre del país.
 * `Country_code_coi`: Código expedido por el Comité Olímpico Internacional.
 * `Num_Summer`: Número de participaciones en Olimpiadas de Verano.
 * `gold_summer`: Número de medallas de oro ganadas en Olimpiadas de Verano.
 * `silver_summer`: Número de medallas de plata ganadas en Olimpiadas de Verano.
 * `bronze_summer`: Número de medallas de bronce ganadas en Olimpiadas de Verano.
 * `Total_summer`: Total de medallas de ganadas en Olimpiadas de Verano.
 * `Num_Winter`: Número de participaciones en Olimpiadas de Invierno.
 * `gold_winter`: Número de medallas de oro ganadas en Olimpiadas de Invierno.
 * `silver_winter`: Número de medallas de plata ganadas en Olimpiadas de Invierno.
 * `bronze_winter`: Número de medallas de bronce ganadas en Olimpiadas de Invierno.
 * `Total_winter`: Total de medallas de ganadas en Olimpiadas de Invierno.
 * `Num_Games`: Total de participaciones.
 * `gold`: Total de medallas de oro ganadas.
 * `silver`: Total de medallas de plata ganadas.
 * `bronze`: Total de medallas de bronce ganadas.
 * `Combined_total`: Total de medallas combinadas.

El dataframe queda de la siguiente manera 

In [10]:
df.sample(10)

,Country,Country_name,Country_code_coi,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
8,Azerbaijan (AZE),Azerbaijan,AZE,8,9,16,31,56,7,0,0,0,0,15,9,16,31,56
65,Iran (IRI),Iran,IRI,18,27,29,32,88,12,0,0,0,0,30,27,29,32,88
43,Ethiopia (ETH),Ethiopia,ETH,15,24,15,23,62,2,0,0,0,0,17,24,15,23,62
40,Egypt (EGY),Egypt,EGY,24,9,12,20,41,1,0,0,0,0,25,9,12,20,41
144,Turkey (TUR),Turkey,TUR,24,41,29,41,111,18,0,0,0,0,42,41,29,41,111
79,Kyrgyzstan (KGZ),Kyrgyzstan,KGZ,8,0,5,8,13,8,0,0,0,0,16,0,5,8,13
23,Canada (CAN),Canada,CAN,28,80,117,156,353,24,77,72,76,225,52,157,189,232,578
13,Belgium (BEL),Belgium,BEL,28,47,57,63,167,22,2,2,4,8,50,49,59,67,175
2,Algeria (ALG),Algeria,ALG,15,7,4,9,20,3,0,0,0,0,18,7,4,9,20
1,Albania (ALB),Albania,ALB,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2


In [11]:
df.shape

(162, 18)

### Dataframe de países

Con la finalidad de conocer un poco mejor a los países, vamos a importar un dataframe con un poco de su información.

Primero vamos con información relevante. Primer crearemos algunas funciones que permitirá hacer la extracción y el tratamiento correspondiente:

In [12]:
def tratamiento_columnas(dataframe):

	columns = dataframe.columns

	new_columns = []
	for col in columns:

		if not isinstance(col, str):
			new_columns.append(col)
			continue

		col = unidecode(col.lower()).strip()
		while "  " in col:
			col = col.replace("  ", " ")
		col = re.sub(r"\s", "_", col)

		if "superficie" in col:
			col = "superficie_km2"
		elif "poblacion" in col:
			col = "poblacion"
		elif "iso" in col:
			col = col.replace("iso", "ISO")
		elif "idh" in col:
			col = col.upper()
		elif col in ["nombre_oficial", "pais", "dependencia", "region_ultraperiferica", "region"]:
			col = "nombre"

		new_columns.append(col)

	dataframe.columns = new_columns

	return dataframe

def validar_continente(pais, series):
	return any(map(lambda x: bool(re.search(pais, x)), series.values))

def titulos_tratamiento(string):
	pattern = re.compile(r"(.*?)(?:\[.+\])+.*(\(.+\))?")

	if (match_ := pattern.fullmatch(string)):
		return match_.group(1)
	else:
		return string

def data_frame_paises():
	mapping = {
		"Alemania" : "Europa",
		"Kosovo" : "Europa", # Estados con reconocimiento limitado
		"China" : "Asia",
		"Israel" : "Asia",  # Estados con reconocimiento limitado
		"Isla de Navidad" : "Asia", # Territorios dependientes
		"Macao": "Asia", # Regiones administrativas especiales de la República Popular de China
		"Angola" : "África",
		"Chile" : "América",
		"Anguila" : "América", # Territorios dependientes
		"Guadalupe" : "América", # Países americanos integrados en estados no americanos
		"Australia" : "Oceanía",
		"Guam" : "Oceanía" # Territorios dependientes
	}

	df_continents = pd.read_html(url_paises)

	tablas = []

	df_tratados = map(tratamiento_columnas, df_continents)

	for tb in df_tratados:
		try:
			map_country = list(filter(lambda x: validar_continente(x, tb["nombre"]), mapping.keys()))[0]
		except (KeyError, IndexError) as e:
			#print(e)
			continue

		tb["Continente"] = mapping[map_country]
		tablas.append(tb.copy())

	df_countries = pd.concat(tablas, axis=0)
	df_countries["nombre"] = df_countries["nombre"].map(titulos_tratamiento)

	return df_countries

def obtener_nombre_en(series):
	pais = pycountry.countries.get(alpha_3=series["ISO_3166-1"])

	if not pais:
		pais = pycountry.historic_countries.get(alpha_3=series["ISO_3166-1"])

	if not pais:
		return np.nan

	return pais.name


def extraer_codigo_iso(series):
	traducidos = {
		'Botsuana': 'Botswana',
		'Comoras': 'Comoros',
		'Gambia': 'Republic of The Gambia',
		'Guinea-Bisáu': 'Guinea-Bissau',
		'Malaui': 'Malawi',
		'Mauricio': 'Mauritius',
		'República Democrática del Congo': 'Democratic Republic of the Congo',
		'República del Congo': 'Republic of the Congo',
		'Santo Tomé y Príncipe': 'São Tomé and Príncipe',
		'Somalilandia (República de Somalilandia)': 'Somaliland',
		'Suazilandia': 'Swaziland',  # Suazilandia cambió su nombre a Eswatini en 2018
		'Sudán del Sur (República de Sudán del Sur)': 'South Sudan',
		'Yibuti': 'Djibouti'
	}

	if isinstance(series["ISO_3166-1"], str):
		if len(series["ISO_3166-1"]) == 3:
			return series["ISO_3166-1"]

	t = Translator()
	pais_ = series.nombre

	pais_ingles = t.translate(pais_, src="es", dest="en")
	countryinfo = CountryInfo(pais_ingles.text)

	try:
		return countryinfo.iso(3)
	except KeyError:
		try:
			countryinfo = CountryInfo(traducidos[pais_])
			return countryinfo.iso(3)
		except KeyError:
			#print(f"No se pudo obtener código ISO de {pais_}")
			return np.nan

In [13]:
df_countries = data_frame_paises()
df_countries = df_countries.dropna(axis=1, how="all")
df_countries = df_countries.rename(columns={"codigo_ISO_3166-1" : "ISO_3166-1"})
df_countries["ISO_3166-1"] = df_countries.apply(extraer_codigo_iso, axis=1)
df_countries[df_countries["ISO_3166-1"].isna()].to_csv(os.path.join(data_path, "info_countries_ISO_nan.csv"), index=False, encoding="utf-8")
df_countries = df_countries.dropna(subset="ISO_3166-1")
df_countries["nombre_en"] = df_countries.apply(obtener_nombre_en, axis=1)
df_countries.sample(10)

,nombre,ISO_3166-1,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente,estatus,idioma_oficial,pib_per_capita_(us$),IDH,tipo_de_dependencia,idiomas,territorio,nombre_en
20,Honduras,HND,112 492,8 893 259,Tegucigalpa,Español,"Lempira(Lps, HNL).",América,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Honduras
28,Laos,LAO,236 800,7 019 073,Vientián,Laosiano,"Kip laosiano(₭, LAK).",Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lao People's Democratic Republic
37,Sahara Occidental,ESH,266 000,587 020,"El Aaiún (declarada), Bir Lehlu (temporal)",NaN,"Peseta saharaui, Dinar argelino",África,NaN,ÁrabeEspañol,-,NaN,NaN,NaN,NaN,Western Sahara
34,Omán,OMN,309 500,3 355 262,Mascate,Árabe,"Rial omaní(ع., OMR).",Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oman
41,Tayikistán,TJK,144 100,8 330 946,Dusambé,Tayiko,Somoni tayiko(TJS),Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tajikistan
2,Kiribati,KIR,811,95 000,Tarawa,Inglés y gilbertés,Dólar de Kiribati y Dólar australiano,Oceanía,NaN,NaN,NaN,Medio,NaN,NaN,NaN,Kiribati
1,Aruba,ABW,180,113 648,Oranjestad,Neerlandés,"Florín arubeño(ƒ, AWG).",América,Nación constitutiva de los Países Bajos,NaN,NaN,NaN,NaN,NaN,NaN,Aruba
0,Afganistán,AFG,652.230,33.332.025,Kabul,PastúnPersa darí,"Afgani afgano(Af, AFN).",Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan
23,Nicaragua,NIC,130 370,7 317 798,Managua,Español,"Córdoba(C$, NIO).",América,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nicaragua
13,Ecuador,ECU,283 561,16 080 778,Quito,Español,"Dólar estadounidense($, USD).",América,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ecuador


Ahora verifiquemos que no haya errores con el código ISO, que debe ser único para cada país

In [14]:
df_countries.groupby(["ISO_3166-1"]).filter(lambda x: x["nombre"].nunique() > 1)

,nombre,ISO_3166-1,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente,estatus,idioma_oficial,pib_per_capita_(us$),IDH,tipo_de_dependencia,idiomas,territorio,nombre_en
41,Tayikistán,TJK,144 100,8 330 946,Dusambé,Tayiko,Somoni tayiko(TJS),Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tajikistan
43,Turkmenistán,TJK,488 100,5 291 317,Asjabad,Turcomano,Manat turcomano(ман. TMT),Asia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tajikistan


Para el día 8 de septiembre del 2024, al parecer existe un error donde Tayiskistán y Turkmenistán tienen el mismo ISO, TJK. Este código, según la ISO 31166-1, debería pertenecerle solo a Tayiskistán. El código ISO de Turkmenistán es TKM, vamos a editar ese registro

In [15]:
df_countries.loc[df_countries.nombre == "Turkmenistán", "ISO_3166-1"] = "TKM"
df_countries.loc[df_countries.nombre == "Turkmenistán", "nombre_en"] = "Turkmenistan"

Dado que el código ISO y el código dado por el COI en ocasiones no coinciden, es necesario tener la comparación entre las dos, para eso usaremos otro dataframe

In [16]:
df_codigos = pd.read_html(url_codigos)[1]
df_codigos.columns = [
	"nombre", "COI", "FIFA", "ISO", "Observaciones"
]
df_codigos["ISO"] = df_codigos["ISO"].dropna()
df_codigos

,nombre,COI,FIFA,ISO,Observaciones
0,Afganistán,AFG,AFG,AFG,NaN
1,Åland,NaN,NaN,ALA,No es un país. Pertenece a Finlandia.
2,Albania,ALB,ALB,ALB,NaN
3,Alemania,GER,GER,DEU,NaN
4,Andorra,AND,AND,AND,NaN
...,...,...,...,...,...
249,Yemen,YEM,YEM,YEM,NaN
250,Yibuti,DJI,DJI,DJI,NaN
251,Wallis y Futuna,NaN,NaN,WLF,No es un país. Pertenece a Francia.
252,Zambia,ZAM,ZAM,ZMB,NaN


Solo queda unir ambos dataframes

In [17]:
df_countries.shape

(229, 16)

In [18]:
df_countries = pd.merge(df_countries, df_codigos[["COI", "FIFA", "ISO"]], how="left", left_on="ISO_3166-1", right_on="ISO").drop(columns=["ISO"])
df_countries = df_countries[['nombre', "nombre_en", 'ISO_3166-1', "COI", "FIFA", 'superficie_km2', 'poblacion', 'capital','idiomas_oficiales', 'moneda', 'Continente']]

In [19]:
df_countries.shape

(229, 11)

El dataframe de paises queda con las siguientes columnas
 * `pais`: Nombre de país en español.
 * `ISO_3166-1`: Código ISO 3166-1 alpha 3 dado al país.
 * `ISO`: 
 * `superficie_km2`: Extensión territorial dado en $m^2$.
 * `poblacion`: Número de habitantes.
 * `capital`: Capital administrativa del país.
 * `idiomas_oficiales`: Idiomas oficiales.
 * `moneda`: Moneda de curso legal.
 * `Continente`: Continente al que pertenecen: América, Asia, Africa, Europa, Oceania.
 * `IDH`: Índice de Desarrollo Humano.

In [20]:
df_countries.to_csv(os.path.join(data_path, "info_countries.csv"), index=False, encoding="utf-8")
df_countries.sample(10)

,nombre,nombre_en,ISO_3166-1,COI,FIFA,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente
64,Chipre,Cyprus,CYP,CYP,CYP,9251,1 205 575,Nicosia,GriegoTurco,"Euro(€, EUR).",Asia
153,Suazilandia,Eswatini,SWZ,SWZ,SWZ,17 364,1 032 000,Lobamba (legislativa)Mbabane (administrativa),NaN,Lilangeni,África
174,Costa Rica,Costa Rica,CRI,CRC,CRC,51 100,4 872 543,San José,Español,"Colón(₡, CRC).",América
137,Mauricio,Mauritius,MUS,MRI,MRI,2040,1 219 220,Port Louis,NaN,Rupia de Mauricio,África
69,Filipinas,Philippines,PHL,PHI,PHI,300 000,102 624 209,Manila,TagaloInglés,"Peso filipino(₱, PHP).",Asia
221,Guam,Guam,GUM,GUM,GUM,549,163 941,Agaña,NaN,NaN,Asia
24,Italia,Italy,ITA,ITA,ITA,301.340,62.007.540,Roma,Italiano,"Euro(€, EUR).",Europa
155,Sudán,Sudan,SDN,SUD,SDN,1 886 068,31 436 378,Jartum,NaN,Libra sudanesa,África
86,Omán,Oman,OMN,OMA,OMA,309 500,3 355 262,Mascate,Árabe,"Rial omaní(ع., OMR).",Asia
45,Suecia,Sweden,SWE,SWE,SWE,450.295,9.880.604,Estocolmo,Sueco,"Corona sueca(kr., SEK).",Europa


### Actualizando dataframe de Medallas

Ahora vamos a complementar el dataframe de medallas con algunos datos de los países. Por supuesto, en las Olimpiadas no solo participan países, sino también algunas organizaciones, sin embargo, no los tomaremos en cuenta.

Comencemos con algo simple: agregando el código ISO y el continente al df de medallas

In [21]:
df = df.merge(df_countries[["ISO_3166-1", "COI", "Continente"]], how="left", left_on="Country_code_coi", right_on="COI").drop(columns=["COI"])
df = df[['Country', 'Country_name','Country_code_coi', "ISO_3166-1", "Continente", 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]
df = df.drop_duplicates()
df.sample(5)

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
35,Cyprus (CYP),Cyprus,CYP,CYP,Asia,12,0,2,0,2,12,0,0,0,0,24,0,2,0,2
59,Great Britain (GBR),Great Britain,GBR,GBR,Europa,30,298,340,343,981,24,12,5,17,34,54,310,345,360,1015
121,Soviet Union (URS),Soviet Union,URS,NaN,NaN,9,395,319,296,1010,9,78,57,59,194,18,473,376,355,1204
163,Yugoslavia (YUG),Yugoslavia,YUG,NaN,NaN,16,26,29,28,83,14,0,3,1,4,30,26,32,29,87
152,Turkey (TUR),Turkey,TUR,TUR,Asia,24,41,29,41,111,18,0,0,0,0,42,41,29,41,111


In [22]:
df.shape

(170, 20)

#### Equipos olímpicos sin continente asignado

Posteriormente, es importante tratar aquellos registros que no pudieron ser clasificados por continentes, o bien, que no obtuvieron un código ISO. Qué son los siguientes

In [23]:
df[df.Continente.isna() | df["ISO_3166-1"].isna()]["Country_name"].tolist()

['Australasia',
 'Bohemia',
 'British West Indies',
 'Czechoslovakia',
 'United Team of Germany',
 'East Germany',
 'West Germany',
 'Kosovo',
 'Netherlands Antilles',
 'Refugee Olympic Team',
 'Russian Empire',
 'Soviet Union',
 'Unified Team',
 'Olympic Athletes from Russia',
 'ROC',
 'Serbia and Montenegro',
 'Virgin Islands',
 'Yugoslavia',
 'Individual Neutral Athletes',
 'Independent Olympic Athletes',
 'Independent Olympic Participants',
 'Mixed team']

Las razones por las cuales no pudieron ser clasificados en un continente es que simplemente no se encontraban en la fuente de `df_countries`, lo que a su vez puede también indicar la razón por la que no están en el ISO 3166-1. Las razones pueden ser la siguientes:
 1. No son países, sino equipos olímpicos: Australasia, United Team of Germany, Refugee Olympic Team, Unified Team, Olympic Athletes from Russia, ROC, Individual Neutral Athletes, Independent Olympic Athletes, Independent Olympic Participants y Mixed team.
 2. Son países con reconocimiento limitado: Kosovo.
 3. Países o territorio que ya existen: Bohemia, British West Indies, Czechoslovakia, East Germany, West Germany, Netherlands Antilles, Russian Empire, Soviet Union, Serbia and Montenegro, Yugoslavia.
 4. Territorio dependientes que participan de forma separada a su país: Virgin Islands.
Dada su condición, no es posible darles un código ISO, sin embargo, es posible clasificar algunos por continentes dada su relación geográfica o cultural. Por lo que la clasificación se hará de la siguiente forma,

In [24]:
map_paises_sin_continente = {
	'Australasia' : "Oceanía", # Dado que fue un equipo conformado por atletas tanto de Australia como de Nueva Zelanda
	'Bohemia' : "Europa", # Dada su posición geográfica: Ocupó el actual territorio de la República Checa
	'British West Indies' : "América", # Aunque fueron británicas, su posición geográfica y cultural la hace más cercana a América
	'Czechoslovakia' : "Europa", # Por su posición geográfica
	'United Team of Germany' : "Europa", # Los atletas eran oriundos ya sea de East Germany o de West Germany
	'East Germany' : "Europa", # Por posición geográfica
	'West Germany' : "Europa", # Por posición geográfica
	'Kosovo' : "Europa", # Por posición geográfica
	'Netherlands Antilles' : "América", # Por posición geográfica
	'Refugee Olympic Team': np.nan, # Sus atletas son de lugares variados
	'Serbia and Montenegro' : "Europa", # Por posición geográfica
	'Virgin Islands' : "América", # Por posición geográfica
	'Yugoslavia' : "Europa", # Por posición geográfica
	'Independent Olympic Athletes' : np.nan, # Sus atletas son de lugares variados
	'Independent Olympic Participants' : np.nan, # Sus atletas son de lugares variados
	'Mixed team' : np.nan # Sus atletas son de lugares variados
}

En este mapeo hacen falta algunos equipos olímpicos: Russian Empire, Soviet Union, Unified Team, Olympic Athletes form Russia, ROC y Individual Neutral Athletes. Estos equipos fueron excluidos por su relación cultural con Rusia y otros países limítrofes, y Rusia tiene el problema ser se un país transcontinental, como algunos otros. Será necesario asignar un solo continente a estos países.

En fin, se realiza la asignación correspondiente en la siguiente linea,

In [25]:
df["Continente"] = df.apply(lambda x: map_paises_sin_continente[x.Country_name] if x.Country_name in map_paises_sin_continente else x["Continente"], axis=1)
df.sample(5)

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
68,India (IND),India,IND,IND,Asia,26,10,10,21,41,11,0,0,0,0,37,10,10,21,41
19,Brazil (BRA),Brazil,BRA,BRA,América,24,40,49,81,170,9,0,0,0,0,33,40,49,81,170
64,Haiti (HAI),Haiti,HAI,HTI,América,17,0,1,1,2,1,0,0,0,0,18,0,1,1,2
4,Armenia (ARM),Armenia,ARM,ARM,Europa,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
36,Czech Republic (CZE),Czech Republic,CZE,CZE,Europa,8,22,22,28,72,8,10,11,13,34,16,32,33,41,106


In [26]:
map_paises_continentes_relacion_rusia = {
	'Russian Empire' : np.nan, # se podrá el mismo continente que Rusia
	'Soviet Union' : np.nan, # se podrá el mismo continente que Rusia
	'Unified Team' : np.nan, # se podrá el mismo continente que Rusia
	'Olympic Athletes from Russia' : np.nan, # se podrá el mismo continente que Rusia
	'ROC' : np.nan, # se podrá el mismo continente que Rusia
	'Individual Neutral Athletes' : np.nan, # Por posición geográfica se podrá el mismo continente que Rusia
}

#### Equipos olímpicos de países transcontinentales

Dado que existen países que abarcan varios continentes, esto se traduce como la existencia de registros casi duplicados en el dataframe de Medallas. Estos registros son idénticos, salvo que difieren en el continente, lo que pone en evidencia su estatus de países transcontinentales. A continuación se listan aquellos que cumplen este criterio,

In [27]:
df[df.duplicated(subset=["Country_code_coi"], keep=False)]

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
4,Armenia (ARM),Armenia,ARM,ARM,Europa,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
5,Armenia (ARM),Armenia,ARM,ARM,Asia,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
9,Azerbaijan (AZE),Azerbaijan,AZE,AZE,Europa,8,9,16,31,56,7,0,0,0,0,15,9,16,31,56
10,Azerbaijan (AZE),Azerbaijan,AZE,AZE,Asia,8,9,16,31,56,7,0,0,0,0,15,9,16,31,56
34,Cyprus (CYP),Cyprus,CYP,CYP,Europa,12,0,2,0,2,12,0,0,0,0,24,0,2,0,2
35,Cyprus (CYP),Cyprus,CYP,CYP,Asia,12,0,2,0,2,12,0,0,0,0,24,0,2,0,2
43,Egypt (EGY),Egypt,EGY,EGY,Asia,24,9,12,20,41,1,0,0,0,0,25,9,12,20,41
44,Egypt (EGY),Egypt,EGY,EGY,África,24,9,12,20,41,1,0,0,0,0,25,9,12,20,41
52,Georgia (GEO),Georgia,GEO,GEO,Europa,8,13,15,19,47,8,0,0,0,0,16,13,15,19,47
53,Georgia (GEO),Georgia,GEO,GEO,Asia,8,13,15,19,47,8,0,0,0,0,16,13,15,19,47


Los países que se muestran en la lista son: Armenia, Azerbaijan, Cyprus, Egypt, Georgia, Kazakhstan, Russia y Turkey. Para lograr una clasificación eficaz se recurrirá a relaciones culturales con cada continente donde su territorio se asienta. La elección hecha será la siguiente,
 * **Armenia**: *Europa*,
 * **Azerbaijan**: *Asia*,
 * **Cyprus**: *Europa*; Aunque geográficamente la mayor parte de su territorio está en Asia, tiene una relación más estrecha con Europa.
 * **Egypt**: *África*,
 * **Georgia**: *Europa*; La disparidad que se produjo al categorizarla en continente se debió a la ambigüedad de su posición geográfica, sin embargo, es más afín a Europa que a Asia.
 * **Kazakhstan**: *Asia*,
 * **Russia**: *Europa*; A pesar de que la mayor parte de su territorio se encuentra en Asia, su centro de poder y su influencia cultural y política se encuentra en Europa.
 * **Turkey**: *Europa*; De la misma forma que Russia, Turkey esta más alineada cultural y políticamente a Europa.

Dado lo anterior, es necesario eliminar los registros que no son de interés, para esto crearemos un mapeo


In [28]:
map_eliminar_registros_casi_duplicados = {
	"Armenia" : "Europa",
	"Azerbaijan" : "Asia",
	"Cyprus" : "Europa",
	"Egypt" : "África",
	"Georgia" : "Europa",
	"Kazakhstan" : "Asia",
	"Russia" : "Europa",
	"Turkey" : "Europa"
}


Con este mapeo vamos a asegurarnos de mantener el registro de interés. Eliminamos los registros que no son de interés

In [29]:
index_eliminar = df[df.apply(lambda x: x["Continente"] != map_eliminar_registros_casi_duplicados[x["Country_name"]] if x["Country_name"] in map_eliminar_registros_casi_duplicados else False, axis=1)].index
df = df.drop(index_eliminar)

dado que la ambigüedad del continente al que pertenece Russia se ha arreglado, es momento de asignar el continente a los equipos olímpicos relacionados con Russia,

In [30]:
map_paises_continentes_relacion_rusia = {
	'Russian Empire' : "Europa",
	'Soviet Union' : "Europa",
	'Unified Team' : "Europa",
	'Olympic Athletes from Russia' : "Europa",
	'ROC' : "Europa",
	'Individual Neutral Athletes' : "Europa"
}
df["Continente"] = df.apply(lambda x: map_paises_continentes_relacion_rusia[x.Country_name] if x.Country_name in map_paises_continentes_relacion_rusia else x["Continente"], axis=1)
df.sample(5)

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
117,Romania (ROU),Romania,ROU,ROU,Europa,23,93,101,123,317,22,0,0,1,1,45,93,101,124,318
32,Croatia (CRO),Croatia,CRO,HRV,Europa,9,16,15,17,48,9,4,6,1,11,18,20,21,18,59
150,Tunisia (TUN),Tunisia,TUN,TUN,África,16,6,4,8,18,0,0,0,0,0,16,6,4,8,18
123,Olympic Athletes from Russia (OAR),Olympic Athletes from Russia,OAR,NaN,Europa,0,0,0,0,0,1,2,6,9,17,1,2,6,9,17
157,United States (USA),United States,USA,USA,América,29,1105,879,780,2764,24,114,121,95,330,53,1219,1000,875,3094


In [31]:
df.Continente.unique()

array(['Asia', 'Europa', 'África', 'América', 'Oceanía', nan],
      dtype=object)

### Exportando el dataframe de Medallas

In [32]:
df.to_csv(os.path.join(data_path,"olympics.csv"), index=False, encoding="utf-8")
df.to_excel(os.path.join(data_path,"olympics_excel.xlsx"), index=False, encoding="utf-8")